# Create an Advanced Pipeline
This pipeline will perform hyper-parameter tuning with Kubeflow Katib, model training with TensorFlow, model deploying with TensorFlow Serving,  model predicting with a sample web-based UI.

In [1]:
!pip3 install Pillow kfp --upgrade --user
!pip install -q awscli

    100% |████████████████████████████████| 2.1MB 22.6MB/s eta 0:00:01
    100% |████████████████████████████████| 122kB 62.4MB/s ta 0:00:01
  Stored in directory: /root/.cache/pip/wheels/39/7a/5f/2f68f60ffe1c09914eedfbf64dc607ed286b63d3e6a4b48893
  Stored in directory: /root/.cache/pip/wheels/d0/85/34/cdf006df2be7a462c85d8760ffea2db4e65c3c6efeb3f2f4ec
  Stored in directory: /root/.cache/pip/wheels/63/e6/37/b9a55c75197247b8244e6564d0117d949d647d2df3294f3a28
Successfully built kfp kfp-server-api strip-hints
  Found existing installation: kfp-server-api 0.1.18.3
    Uninstalling kfp-server-api-0.1.18.3:
      Successfully uninstalled kfp-server-api-0.1.18.3
  The script strip-hints is installed in '/root/.local/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  The scripts dsl-compile and kfp are installed in '/root/.local/bin' which is not on PATH.
  Consider adding this directory to PATH or, i

You are using pip version 19.0.1, however version 20.1b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.1, however version 20.1b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
# Restart the kernel to pick up pip installed libraries
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [1]:
%%bash

echo "Region: $(curl -s http://169.254.169.254/latest/meta-data/placement/availability-zone | sed 's/\(.*\)[a-z]/\1/')"

Region: us-west-2


In [2]:
import boto3

#################################
#################################
# REPLACE AWS_REGION= with the current region
#  surround with single quotes
AWS_REGION='us-west-2'
#################################
#################################

AWS_ACCOUNT_ID=boto3.client('sts').get_caller_identity().get('Account')

S3_BUCKET='sagemaker-{}-{}'.format(AWS_REGION, AWS_ACCOUNT_ID)
print('S3 Bucket: {}'.format(S3_BUCKET))

S3 Bucket: sagemaker-us-west-2-903253828154


In [3]:
import random
import string
from src.mnist.src import katib_launch_args, converter, resource_provider, tfjoblaunch_args_provider
import kfp
from kfp import components
from kfp.components import func_to_container_op
import kfp.dsl as dsl

# Setup S3 Bucket

In [4]:
mnist_bucket=S3_BUCKET
s3_bucket_path='s3://{}'.format(mnist_bucket)

print(s3_bucket_path)

s3://sagemaker-us-west-2-903253828154


# Build Kubeflow Pipeline
Note:  This pipeline is hard-coded to `us-west-2`

In [5]:
namespace='kubeflow'
HASH = ''.join([random.choice(string.ascii_lowercase) for n in range(16)] + [random.choice(string.digits) for n in range(16)])

@dsl.pipeline(
    name="End to end pipeline",
    description="An end to end example including hyperparameter tuning, train and inference."
)
def mnist_pipeline(
        name="mnist-{{workflow.uid}}",
        namespace=namespace,
        step="1000",
        s3bucketexportpath="",
        ttlSecondsAfterFinished=-1,
        tfjobTimeoutMinutes=60,
        deleteAfterDone=False):

    # step 1: create a Katib experiment to tune hyperparameters
    objectiveConfig, algorithmConfig, parameters, trialTemplate, metricsCollectorSpec =  \
        katib_launch_args.argugments_provide(objective_type="minimize",
                                             objective_goal=0.001,
                                             objective_metrics="loss",
                                             algorithm="random",
                                             parameters_lr_min="0.01",
                                             parameters_lr_max="0.03",
                                             parameters_batchsize=["16", "32", "64"],
                                             tf_train_steps="200",
                                             image="chuckshow/mnist-tf-pipeline:latest",
                                             worker_num=3)
                          

    katib_experiment_launcher_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/master/components/kubeflow/katib-launcher/component.yaml')
    op1 = katib_experiment_launcher_op(
            experiment_name=name,
            experiment_namespace=namespace,
            parallel_trial_count=3,
            max_trial_count=12,
            objective=str(objectiveConfig),
            algorithm=str(algorithmConfig),
            trial_template=str(trialTemplate),
            parameters=str(parameters),
            metrics_collector=str(metricsCollectorSpec),
            delete_finished_experiment=False)
    
    # step 1.5: convert Katib best parameteres into string
    convert_op = func_to_container_op(converter.convert_mnist_experiment_result)
    op2 = convert_op(op1.output)
    
    # step2: create a TFJob Launcher to train your model with best hyperparameter tuned by Katib
    tfjob_launcher_op = components.load_component_from_file("./src/mnist/launcher/component.yaml")
    
    chief, worker = tfjoblaunch_args_provider.tfjoblauncher_args(step=step, 
                                                                 s3bucketexportpath=s3bucketexportpath, 
                                                                 args=op2.output,
                                                                 aws_region=AWS_REGION)
    
    train = tfjob_launcher_op(
        name=name,
        namespace=namespace,
        ttl_seconds_after_finished=ttlSecondsAfterFinished,
        worker_spec=worker,
        chief_spec=chief,
        tfjob_timeout_minutes=tfjobTimeoutMinutes,
        delete_finished_tfjob=deleteAfterDone,
    )
    
    # step 3: model inferencese by Tensorflow Serving    
    servingdeploy_name = 'mnist-model' + HASH
    
    deploy = resource_provider.tfservingdeploy_resource(namespace=namespace,
                                                       s3bucketexportpath=s3bucketexportpath,
                                                       servingdeploy_name=servingdeploy_name,
                                                       aws_region=AWS_REGION)
    
    deployment = dsl.ResourceOp(
        name="deploy",
        k8s_resource=deploy,
    ).after(train)
    
    servingsvc_name = 'mnist-service'
    serviceresource = resource_provider.tfservingsvc_resource(namespace=namespace,
                                                             servingdeploy_name=servingdeploy_name,
                                                             servingsvc_name=servingsvc_name)
    
    service = dsl.ResourceOp(
        name="service",
        k8s_resource=serviceresource
    ).after(deployment)
    

    # step 4: mnist ui deploy
    ui_name = 'mnist-ui' + HASH
    uideployresource = resource_provider.uideploy_resource(namespace=namespace,
                                                          ui_name=ui_name)
    
    uideploy = dsl.ResourceOp(
        name="uideploy",
        k8s_resource=uideployresource
    ).after(train)
   
    uiserviceresource = resource_provider.uisvc_resource(namespace=namespace,
                                                        ui_name=ui_name)
    
    uiservice = dsl.ResourceOp(
        name="uiservice",
        k8s_resource=uiserviceresource
    ).after(uideploy)
    
    uivirtualserviceresource = resource_provider.uivirtualsvc_resource(namespace=namespace,
                                                                      ui_name=ui_name)
    
    uivirtualservice = dsl.ResourceOp(
        name="uivirtualservice",
        k8s_resource=uivirtualserviceresource
    ).after(uiservice)
    

### Submit the pipeline

In [6]:
pipeline = kfp.Client().create_run_from_pipeline_func(mnist_pipeline, arguments={"s3bucketexportpath":'{}/export'.format(s3_bucket_path)})

/root/.local/lib/python3.6/site-packages/kfp/components/_data_passing.py:168: UserWarning: Missing type name was inferred as "Integer" based on the value "-1".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))
/root/.local/lib/python3.6/site-packages/kfp/components/_data_passing.py:168: UserWarning: Missing type name was inferred as "Integer" based on the value "60".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))
/root/.local/lib/python3.6/site-packages/kfp/components/_data_passing.py:168: UserWarning: Missing type name was inferred as "Boolean" based on the value "False".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))
/root/.local/lib/python3.6/site-packages/kfp/components/_data_passing.py:189: UserWarning: There are no registered serializers from type "bool" to type "Bool", so the value will 

# Navigate to the Experiment and Run links ^^ above - and wait for the pipeline to complete before continuing!!

_Otherwise, you will see errors below!!_

### Invoke serving API via Python client

In [7]:
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import os
import subprocess
import argparse

import random
import json
import requests

# Note:  Port 8500 is HTTP/REST API.
endpoint = "http://mnist-service.{}.svc.cluster.local:8500/v1/models/mnist:predict".format(namespace)

# Prepare test dataset
fashion_mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# scale the values to 0.0 to 1.0
train_images = train_images / 255.0
test_images = test_images / 255.0

# reshape for feeding into the model
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

class_names = ['0','1','2','3','4','5','6','7','8','9']

# Random generate one image
rando = random.randint(0,len(test_images)-1)
data = json.dumps({"signature_name": "serving_default", "instances": test_images[rando:rando+1].tolist()})
print('Data: {} ... {}'.format(data[:50], data[len(data)-52:]))

# HTTP call
headers = {"content-type": "application/json"}
json_response = requests.post(endpoint, data=data, headers=headers)
predictions = json.loads(json_response.text)['predictions']

print(predictions)

title = 'The model thought this was a class {}, and it was actually a class {}'.format(
test_labels[rando], predictions[0]['classes'])
print('\n')
print(title)

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

11493376/11490434 [==============================] - 0s 0us/step
Data: {"signature_name": "serving_default", "instances": ...  [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]]]}


ConnectionError: HTTPConnectionPool(host='mnist-service.kubeflow.svc.cluster.local', port=8500): Max retries exceeded with url: /v1/models/mnist:predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fdd99abc080>: Failed to establish a new connection: [Errno -2] Name or service not known',))

### Invoke Serving API through a Web UI
Navigate to the following URL to make predictions using the sample application.  

_Replace the **[istio-endpoint]** below with the same **hostname** of the browser url for this current Kubeflow notebook.  Just the **hostname**._

```
http://<istio-endpoint>/mnist/kubeflow/ui/ <== REQUIRES TRAILING SLASH
```

_Replace the **[istio-endpoint]** above with the same **hostname** of the browser url for this current Kubeflow notebook.  Just the **hostname**._  

### (Optional) If You Re-Run This Notebook, You Will First Need to Delete Some Resources
Note:  If you re-run this notebook, you will first need to delete the existing `mnist-service` using the following command **in your SageMaker notebook** (not this Kubeflow notebook - you won't have access.):

In [ ]:
# mnistuisvc_name = "mnist-ui" + HASH
# mnistmodel_name= "mnist-model" + HASH

# print('This is not required, but if you want to clean up the resources, you would execute the following commands in the SageMaker notebook - not in this Kubeflow notebook')
# print('')
# print('kubectl delete svc -n kubeflow mnist-service')
# print('kubectl delete svc -n kubeflow {}'.format(mnistuisvc_name))
# print('kubectl delete deploy -n kubeflow {}'.format(mnistmodel_name))
# print('kubectl delete deploy -n kubeflow {}'.format(mnistuisvc_name))
# print('kubectl delete virtualservice -n kubeflow {}'.format(mnistuisvc_name))